# Chapter 6. 요약

In [1]:
import sys
print(sys.version)

3.10.11 (main, Apr 20 2023, 19:02:41) [GCC 11.2.0]


In [2]:
!nvidia-smi

Sun Jul  2 15:40:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   39C    P5    23W / 170W |    504MiB / 12288MiB |     23%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# GPU 메모리 사용량 확인
if device.type =="cuda":
    allocated_memory = torch.cuda.memory_allocated(device = device)
    cached_memory = torch.cuda.memory_reserved(device = device)

print(f"Allocated Memory:{allocated_memory / 1024 ** 2:.2f} MB")
print(f"Cached Memory: {cached_memory / 1024**2:.2f} MB")

Allocated Memory:0.00 MB
Cached Memory: 0.00 MB


In [28]:
# colab 수행 시 
# !pip install transformers
# !pip install datasets

**(서론)**

언젠가 한 번쯤 문서를 요약해야 할 때가 있습니다. 요약할 문서는 연구 논문이나 재무 실적 보고서 아니면 이메일 스레드일지도 모릅니다. 생각해보면 이런 작업에는 긴 단락을 이해하고, 관련 내용을 추론하고, 원래 문서의 주제를 통합해 유창한 텍스트를 생성하는 등 다양한 능력이 필요합니다. 또 기사를 정확하게 요약하는 방법과 법률 계약서를 요약하는 방법은 매우 다르기 때문에 정교한 수준의 도메인 일반화가 필요합니다. 이런 이유로 트랜스포머를 포함한 자연어 모델에게 텍스트 요약은 어려운 작업니다. 이런 어려움에도 불구하고 텍스트 요약은 도메인 전문가의 작업 속도를 크게 높이고 기업에서 내부 지식을 집약하고, 계약을 요약하고 소셜 미디어를 위한 자동 콘텐츠를 생성하는 등의 작업에 사용됩니다. 


이와 관련된 도전 과제를 이해하기 위해 이 장은 사전 훈련된 트랜스포머를 사용해 문서를 요약하는 방법을 알아보겠습니다. 요약은 입력과 출력이 텍스트인 고전적인 시퀀스-투-시퀀스 작업니다. 1장에서 보았듯이 요약에는 인코더-디코더 트랜스포머가 잘 맞습니다. 


이 장에서는 인코더-디코더 모델을 만들어 여러 사람이 주고 받은 대화를 간결하게 요약하겠습니다. 

하지만 그 전에 요약에 사용하는 대표적인 데이터 셋인 CNN/DAilyMail 말뭉치를 살펴보겠습니다.

## 6.1 CNN/DailyMail 데이터셋

CNN/DailyMail 데이터셋은 300,000개 뉴스 기사와 요약의 쌍으로 구성됐습니다. 요약은 CNN과 DailyMail이 기사에 첨부한 글머리 목록의 내용인데, 요약이 본문에서 추출되지 않고 추상적이라는 중요한 특징이 있습니다. 즉, 단순한 발췌가 아니라 새로운 문장으로 구성됐다는 이야기입니다. 이 데이터 셋은 허깅 페이스 허브(https://huggingface.co/datasets/cnn_dailymail)에서 제공합니다. 여기서는 요약을 위해 익명화 처리를 하지 않은 3.0.0 버전을 사용하겠습니다. 4장에서 서브셋을 선택한 대와 비슷한 방식으로 version 매개변수를 사용해 버전을 선택합니다. 

In [4]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", version = "3.0.0")
print(f"특성: {dataset['train'].column_names}")

Found cached dataset cnn_dailymail (/home/bread/.cache/huggingface/datasets/cnn_dailymail/default/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

특성: ['article', 'highlights', 'id']


In [5]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# GPU 메모리 사용량 확인
if device.type =="cuda":
    allocated_memory = torch.cuda.memory_allocated(device = device)
    cached_memory = torch.cuda.memory_reserved(device = device)

print(f"Allocated Memory:{allocated_memory / 1024 ** 2:.2f} MB")
print(f"Cached Memory: {cached_memory / 1024**2:.2f} MB")

Allocated Memory:0.00 MB
Cached Memory: 0.00 MB


In [6]:
import psutil

# 현재 프로세스의 메모리 정보 얻기
process = psutil.Process()
print(f"result : {process.memory_info().rss / 10**6:.2f} MB")  # 현재 사용 중인 메모리 양 (Resident Set Size)

result : 464.91 MB


이 데이터 셋은 세가지 특성이 있습니다. 

 - 뉴스 기사를 담은 article
 
 - 요약에 해당하는 highlights
 
 - 기사의 고유 아이디 id

In [7]:
# 기사 중 하나의 내용을 발췌함

sample = dataset["train"][1]
print(f""" 기사 (500개 문자 발췌, 총 길이: {len(sample["article"])}):""")
print(sample["article"][:500])
print(f'\n 요약 (길이: {len(sample["highlights"])}):')
print(sample["highlights"])

 기사 (500개 문자 발췌, 총 길이: 4051):
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

 요약 (길이: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


기사가 요약에 비해 매우 긴 경우도 있습니다. 이 경우 17배나 차이가 납니다. 대부분 트랜스포머 모델의 문맥 크기가 몇 단락에 해당하는 분량인 1,000개 토큰 정도로 제한되므로, 긴 기사는 트랜스포머 모델에 문제를 일으깁니다. 이를 처리하는 표준적이면서 가장 단순한 방법은 모델의 문맥 크기에 맞춰 텍스트를 자른 것입니다. 텍스트 끝부분에 중요한 정보가 있다면 사라지겠지만, 이는 모델 구조의 제약으로 생기는 불가피한 선택입니다.

## 6.2 텍스트 요약 파이프라인

앞의 예제 기사에 대한 출력을 정성적으로 살펴보면서 요약 작업에 많이 사용되는 트랜스포머 모델 몇 가지를 알아보겠습니다. 

살펴볼 모델 구조는 최대 입력 크기가 각각 다르지만 동일한 입력을 사용하고 출력을 비교하기 위해 입력 텍스트를 2,000자로 제한하겠습니다.

In [8]:
sample_text = dataset["train"][1]["article"][:2000]

# 딕셔너리에 각 모델이 생성한 요약을 저장합니다.
summaries = {}

요약에서는 관례적으로 요약 문장을 줄바꿈으로 나눕니다. 마침표마다 그 뒤에 줄바꿈 토큰을 추가해도 되지만 그러면 'U.S'나 'U.N' 같은 문자열을 처리하지 못합니다. 

**NLTK(Natural Language Toolkit)** 패키지에는 문장의 종결과 약어에 등장하는 구두점을 구별하는 더 정교한 알고리즘이 있습니다.

In [26]:
import numpy as np

In [27]:
import nltk as nltk
from nltk.tokenize import sent_tokenize

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [ ]:
nltk.download("punkt")

string = "The U.S. are a country. The U.N. is an organization."

sent_tokenize(string)

*에러 발생*

**WARNING** 다음절에서는 여러 개의 대규모 모델을 로드합니다. 메모리가 부족하다면 큰 모델을 작은 모델(가령 "gpt", "t5-small"로 바꾸거나 이 절을 건너뛰고 6.5절 'CNN/DailyMail' 데이터셋에서 pegasus 평가하기로 이동하세요.

### 6.2.1 요약 기준 모델

기사를 요약하는 일반적인 기준 모델(baseline)은 단순히 기사에서 맨 처음 문장 세개를 선택하는 것입니다. 이런 기준 모델은 NLTK 문장 토크나이저로 쉽게 구현할 수 있습니다.

In [25]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

summaries["baseline"] = three_sentence_summary(sample_text)

NameError: name 'sent_tokenize' is not defined

### 6.2.2 GPT-2

5장에서 GPT-2가 주어진 프롬프트로 텍스트를 생성하는 방법을 보았습니다. 이 모델은 입력 텍스트 뒤에 'TL;DR'을 추가해 요약을 생성하는 놀라운 기능을 발휘합니다. 너무 길어 읽지 않았다.(too long:didn't read)는 문구의 약어 'TL;DR'은 레딧(reddit) 같은 사이트에서 긴 포스트를 짧게 요약할 때 종종 사용됩니다. 허깅페이스 트랜스포머스의 pipeline() 함수로 원본 논문의 방식을 재현하며 요약 작업을 실험해 보겠습니다. 텍스트 생성 파이프라인을 만들고 대용량 GPT-2 모델을 로드합니다. 

In [29]:
from transformers import pipeline, set_seed

set_seed(42) 
pipe = pipeline("text-generation", model = "gpt2-xl")
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length = 512, clean_up_tokenization_spaces = True)
summaries["gpt2"] = "\n".join(sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))

2023-07-02 16:41:52.482451: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-02 16:41:52.692703: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-02 16:41:53.224118: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
module 'numpy' has no attribute '_no_nep50_warning'

나중의 비교를 위해 출력에서 입력 텍스트의 다음 부분을 요약으로 추출해 파이썬 딕셔너리에 저장합니다.

### 6.2.3 T5

다음에 시도할 모델은 T5 트랜스포머입니다. 3장에서 보았듯이, 이 모델의 개발자들은 NLP에서 포괄적인 전이 학습 연구를 수행해 모든 작업을 텍스트-투-텍스트 작업으로 구성하는 범용의 트랜스포머 아키텍처를 만들었습니다. 

T5 체크포인트는 요약을 포함해 여러 작업에서 (마슷킹된 단어를 재구성하기 위한) 비지도 학습 데이터와 지도 학습 데이터를 섞은 데이터로 훈련됐습니다. 따라서 미세 튜닝 없이 이 체크포인트를 사전 훈련에 썼던 것과 동일한 프롬프트를 사용해 바로 요약에 사용할 수 있습니다. 

문서 요약에 사용할 모델의 입력 포맷은 "`summarize: <ARTICLE>`" 이고, 번역에 사용할 입력 포맷은 "`translate English to German: <TEXT>`" 입니다. 
    
[그림 6-1]에서 보듯이 이런 입력 포맷으로 T5는 많은 작업을 해결하는 매우 다재다능한 모델입니다. 
    
요약을 위해 pipeline() 함수로 T5를 바로 로드하겠습니다. 이 함수는 입력을 text-to-text 포맷으로 처리하므로 앞에 "summarize"를 붙일 필요가 없습니다.

In [30]:
pipe = pipeline("summarization", model = "t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

NameError: name 'pipeline' is not defined

![그림6-1](image/chapter06_01.png)

### 6.2.4 BART

BART도 인코더-디코더 구조를 사용하는 모델로, 손상된 입력을 재구성하도록 훈련됐습니다. 이를 위해 BERT와 GPT-2의 사전 훈련 방식을 결합합니다. 여기서는 특별히 CNN/DailyMail 데이터셋에 미세 튜닝된 facebook/bart-large-ccn 체크포인트를 사용하겠습니다.

In [31]:
pipe = pipeline("summarization", model = "facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

NameError: name 'pipeline' is not defined

### 6.2.5 PEGASUS

PEGASUS는 BART와 마찬가지로 인코더-디코더 트랜스포머입니다. [그림6-2]대로 이 모델은 여러 문장으로 구성된 텍스트에서 마스킹된 문장을 예측하는 사전 훈련 목표로 훈련되었습니다. 논문의 저자들은 사전 훈련 목표가 후속 작업에 가까울수록 더 효과적이라고 주장합니다. 일반적인 언어 모델링보다 요약에 특화된 사전 훈련 목표를 찾기 위해 대규모 말뭉치에서 (내용 중복을 측정하는 요약 평가 지표를 사용해) 주변 문단의 내용을 대부분 담은 문장을 자동으로 식별했습니다. 이런 문장을 재구성하도록 PEGASUS 모델을 사전 훈련해 최고 수준의 텍스트 요약 모델을 얻었습니다.

![그림6-2](image/chapter06_02.png)

이 모델은 줄 바꿈하는 특수 토큰이 있으므로 sent_tokenize() 함수를 사용할 필요가 없습니다. 

In [32]:
pipe = pipeline("summarization", model = "google/pegasus-cnn-dailymail")
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

NameError: name 'pipeline' is not defined

## 6.3 요약 결과 비교하기

각기 다른 네 가지 모델로 요약을 생성했습니다. 결과를 비교해보죠. 한 모델(GPT-2)은 데이터셋에서 전혀 훈련되지 않았다는 점을 기억해 두세요. 한 모델 (T5)은 여러 작업 중의 하나로 이 작업을 위해 미세 튜닝되었습니다. 두 모델(BART와 PEGASUS)은 이 작업만을 위해 미세 튜닝되었습니다. 

네 모델이 생성한 요약 결과를 확인해 보죠.

In [33]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .



--- 책 원문 내용 ---
   
모델 출력에서 가장 먼저 눈에 띄는 것은 GPT-2가 생성한 요약이 다른 결과와 크게 다른 것입니다. 

텍스트를 요약하는 대신 등장인물을 요약했습니다. 

GPT-2 모델은 진짜 요약을 생성하도록 명시적으로 훈련되지 않았기 때문에 종종 사실을 지어내거나 환상을 만들어 냅니다. 

예를 들어 이 글을 쓰는 시점에 Nesta의 순위는 1위가 아니라 9위 입니다. 

정답 요약과 다른 세 모델의 요약을 비교하면 놀랄 정도로 많이 중복되면 그 중 PEGASUS 출력과 가장 비슷합니다.

다음 절에서 생성된 텍스트의 품질을 측정하기 위해 개발된 일반적인 지표를 알아보겠습니다. 

### 6.4 생성된 텍스트 품질 평가하기

평가 지표는 모델을 훈련할 때만이 아니라 나중에 제품 환경에서도 모델 성능을 평가하기 때문에 중요합니다. 평가 지표가 나쁘면 모델의 성능 저하를 눈치 채지 못하고, 평가 지표가 비즈니스 목표에 맞지 않으면 어떤 가치도 창출하지 못합니다. 

텍스트 생성 작업의 성능 측정은 감성 분석이나 개체명 인식 같은 표준적인 분류 작업만큼 쉽지 않습니다. 

생성된 텍스트를 평가하는 데 가장 널리 사용되는 두 지표는 BLEU와 ROUGE 입니다. 어떻게 정의되는지 알아보죠!

### 6.4.1 BLEU

BLEU(BiLingual Evaluation Understudy)의 개념은 단순합니다. 

  - 생성된 텍스트에서 얼마나 많은 토큰이 참조 테스트 토큰과 완벽하게 똑같이 정렬됐는지 확인하는 대신, 단어 또는 n-그램을 체크합니다. 

  - BLEU는 정밀도를 근간으로 하는 지표입니다.
  
  - 두 텍스트를 비교할 때 참조 텍스트에 있는 단어가 생성된 텍스트에 얼마나 자주 등장하는 지 카운트합니다. 그 후에 생성된 텍스트 길이로 나눕니다.

하지만 이런 단순한 정밀도에는 문제가 있습니다. 생성된 텍스트에 동일 단어가 반복되고 이 단어가 참조 텍스트에 등장한다고 해보죠. 참조 텍스트 길이만큼 반복된다면 정밀도는 완벽합니다. 이런 이류도 BLEU 논문 저자들은 약간의 변화를 주었습니다. 단어를 참조 텍스트에 등장한 횟수만큼만 카운트합니다.

이를 설명하기 위해 참조 텍스트가 'the cat is on the mat'이고 생성된 텍스트가 'the the the the the the'라고 가정해보죠.

이 예시에서 정밀도는 이렇게 계산됩니다.

$$ P_{vanilla} = \frac{6}{6} $$

$$ P_{mod} = \frac{2}{6} $$

간단한 수정으로 훨씬 합리적인 값을 얻었습니다. 이제 이를 확장해 단어 하나만이 아니라 n-그램도 확인할 수 있습니다. 

생성된 텍스트가 snt와 참조 문장 snt'를 비교한다고 해보죠. 특정 n에 대해 가능한 모든 n-그램을 추출해 정밀도를 계산합니다.

$$ p_n = \frac{\sum_{n-gram \in snt'}Count_{clip}(n-gram)}{\sum_{n-gram \in snt}Count(n-gram)} $$

**반복적인 생성에 보상을 주지 않도록 분자의 카운트는 클리핑합니다.**

 - 생성된 문장에서 n-그램의 등장 횟수를 카운트하는 것이 참조 문장에 나타난 횟수로 제한된다는 의미입니다. 

 - 이 식에서 문장의 정의는 그다지 엄격하지 않습니다. 여러 문장에 걸쳐 생성된 텍스트가 있다면 이를 하나의 문장으로 다룹니다.

일반적으로 테스트 세트는 평가할 샘플이 하나 이상 있으니 말뭉치 C에 있는 모든 샘플을 더하도록 이 식을 조금 확장할 필요가 있습니다.

$$ p_n = \frac{\sum_{snt' \in C}\sum_{n-gram \in snt'} Count_{clip}(n-gram)}{\sum_{snt\in C}\sum_{n-gram \in snt} Count(n-gram) } $$

거의 다 왔습니다. 재현율을 고려하지 않기 때문에 짧지만 정밀하게 생성된 시퀀스가 긴 문장보다 유리합니다. 따라서 짧게 생성된 텍스트의 정밀도 점수가 더 좋습니다. 이를 보상하기 위해 BLEU 논문의 저자들은 **브레비티 패널치(Brevity penalty)** 라는 추가 항을 도입했습니다.

$$ BR = min(1, e^{1-l_{ref} \; / \; l_{gen}}) $$

```
LaTeX에서 띄어쓰기를 하는 방법은 4가지가 있다.

\, : 한칸 띄어쓰기.
\; : 두칸 띄어쓰기.
\quad : 네칸(= tab) 띄어쓰기.
\qquad : 여덟칸(= tab*2) 띄어쓰기.
이는 Mathjax에서도 똑같이 활용할 수 있다.
```

최솟값을 선택하므로 이 패널티는 절대 1을 넘지 않고, 생성된 텍스트의 길이 $l_{gen}$ 가 참조 텍스트 $l_{ref}$ 보다 더 작을 때 지수 항이 기하급수적으로 작아집니다. 이 시점에서 왜 재현율도 고려하는 F1-score 같은 기준을 사용하지 않는지 궁금할 겁니다. 그 이유는 번역 데이터셋에는 하나가 아니라 여러 개의 참조 문장이 있는 경우가 있기 때문입니다. 

재현율을 측정하면 전체 참조 문장에 있는 단어를 모두 사용하는 번역에 인센티브가 주어집니다. 따라서 번역의 정밀도가 높고 번역과 참조 문장의 길이가 비슷한지 확인하는 것이 좋습니다. 

마지막으로 모든 것을 합쳐서 BLEU 점수를 계산하는 공식을 만들겠습니다.

$$ BLEU -N = BR \times (\prod_{n=1}^{N}P_n)^{1/N}$$

마지막 항은 1에서 N까지 n-그램에서 수정 정밀도의 기하 평균입니다. BLEU-4 점수가 실제로 많이 사용됩니다. 하지만 이 지표에는 많은 제약이 있습니다. 한 예로, 동의어를 고려하지 않습니다. 유도된 식의 많은 단계가 임시 방편이고 깨지기 쉽습니다. BLEU의 단점을 잘 설명한 레이첼 타트만(Rachael Tatman)의 블로그 포스트 'Evaluating Text Output in NLP:BLEU at Your Own Risk'(https://towardsdatascience.com/evaluating-text-output-in-nlp-bleu-at-your-own-risk-e8609665a213)를 참고하세요.

텍스트 생성 분야에서는 계속해서 더 좋은 평가 지표를 찾고 있습니다. BLEU 같은 지표의 단점을 극복하는 방법을 찾는 연구가 활발히 진행 중입니다. BLEU 지표의 또 다른 약점은 토큰화된 텍스트를 기대한다는 점입니다. 만약 텍스트 토큰화를 정확히 같은 방법으로 하지 않으면 결과가 달라집니다. SacreBLEU는 토큰화 단계를 내재화해 이 문제를 해결합니다. 이 때문에 벤치마킹에서는 이 지표를 선호합니다. 

지금까지 이론적인 면을 살펴봤지만, 우리는 생성된 텍스트에 대해서 실제점수를 계산해야 합니다. 이 로직을 모두 파이썬으로 구현해야 할까요? 걱정하지마세요. 허깅페이스 데이터셋은 측정 지표도 제공합니다! 지표를 로딩하는 방법은 데이터셋을 로딩하는 방법과 비슷합니다. 

In [34]:
from datasets import load_metric

bleu_metric = load_metrci("sacrebleu")

NameError: name 'load_metrci' is not defined

bleu_metric 객체는 Metric 클래스의 인스턴스로 하나의 수집기(aggregator)처럼 작동합니다. add() 메서드에 샘플 하나를 추가하거나 add_batch() 메서드로 배치 전체를 추가합니다. 평가하려는 샘플을 모두 추가한 다음 compute() 메서드를 호출하면 지표가 계산됩니다. 이 메서드는 몇 개의 값으로 구성된 딕셔너리를 반환합니다. 각 n-그램에 대한 정밀도, 길이 페널티, 최종 BLEU 점수 등입니다. 앞에서 예로 든 문장을 사용해보죠.

In [35]:
import pandas as pd
import numpy as np 

bleu_metric.add(prediction = "the the the the the the", reference = ["the cat is on the mat"])
results = bleu_metric.compute(smooth_method = "floor", smooth_value = 0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient = "index", columns = ["Value"])

NameError: name 'bleu_metric' is not defined

**NOTE** BLEU 점수는 여러 참조 번역이 있는 경우에도 계산됩니다. 이 때문에 reference 매개변수에 리스트를 전달합니다. BLEU는 정밀도 계산을 조금 바꿔 n-그램이 하나도 없을 때 최종 점수가 0이 되는 경우를 방지합니다. 이를 위한 방법으로 분자에 상수 값을 추가합니다. 이렇게 하면 n-그램이 없어도 점수가 0이 되지 않습니다. 이를 위한 방법으로 분자에 상수 값을 추가합니다. 이렇게 하면 n-그램이 없어도 점수가 0이 되지 않습니다. 이 값을 설명하기 위해 smooth_value = 0로 지정해 해당 기능을 껐습니다. 

- smooth_method가 floor이면 smooth_value의 기본값이 0.1이고, n-그램이 없을 경우 분자로 0.1을 사용합니다. smooth_method가 add-k이면 smooth_value의 기본값이 1이고, n-그램이 없을 경우 분모와 분자에 1이 더해집니다. smooth_method의 기본값은 'exp'이며 smooth_value를 사용하지 않고, n-그램이 없을 때마다 2의 거듭제곱을 누적해 분모에 곱한 역수를 계산합니다. 

1- 그램의 정밀도는 실제로 2/6 입니다. 반면 2/3/4-그램의 정밀도는 모두 0입니다. counts와 bp 같은 개별 지표의 자세한 내용은 SacreBLEU 저장소(https://github.com/mjpost/sacrebleu)를 참고하세요. 그러면 기하 평균이 0이되므로 BLEU 점수도 0이 됩니다. 정밀도가 매우 높은 또 다른 예시를 확인해보죠.

In [36]:
bleu_metric.add(prediction = "the cat is on mat", reference = ["the cat is on the mat"])
results = bleu_metric.compute(smooth_method = "floor", smooth_value = 0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient = "index", columns = ["Value"])

NameError: name 'bleu_metric' is not defined

BLEU 점수는 텍스트 평가에 널리 사용됩니다. 가능하고 적합한 단어를 모두 포함하는 번역보다 정확한 번역이 선호되기 때문에 특히 기계 번역에 많이 쓰입니다. 

이와 상황이 다른 요약 같은 애플리케이션이 있습니다. 이때는 중요한 정보가 생성된 텍스트에 모두 포함되어야 하므로 높은 재현율이 선호됩니다. 이런 작업에는 주로 ROUGE(Recall-Oriented Understudy for Gisting Evaluation)가 사용됩니다.